# Kuis 3

## Mencari jarak antar 2 bidang

### Cara Manual

In [43]:
import numpy as np
import re
from sympy import symbols, Eq, sympify

In [44]:
def parse_equations(equations):
    variables = set(re.findall(r'[a-zA-Z]', ' '.join(equations)))
    variables = sorted(variables)
    symbols_dict = {var: symbols(var) for var in variables}
    symbol_list = [symbols_dict[var] for var in variables]
    coefficients = []
    constants = []

    for eq in equations:
        lhs, rhs = eq.split('=')
        lhs = sympify(lhs.strip(), locals=symbols_dict)
        rhs = sympify(rhs.strip(), locals=symbols_dict)
        equation = Eq(lhs, rhs)

        coeff_row = [float(lhs.coeff(var).evalf()) for var in symbol_list]
        coefficients.append(coeff_row)

        constant = (rhs - lhs.subs({var: 0 for var in symbol_list})).evalf()
        constants.append(float(constant))

    A = np.array(coefficients, dtype=float)
    B = np.array(constants, dtype=float).flatten()

    return A, B

In [45]:
equation_1 = [
    "5*x - 7*y + 9*z = 34"
]

equation_2 = [
    "-15*x + 21*y - 27*z = 67"
]

A1, B1 = parse_equations(equation_1)
A2, B2 = parse_equations(equation_2)
titik_sample = np.array([5, 0, 1])

### Cara Manual

In [46]:
def jarak_bidang(v1, v2, c1, c2, titik):
    jarak_1 = np.sum(v1[0,0]*titik[0] + v1[0,1]*titik[1] + v1[0,2]*titik[2] - c1[0])
    jarak_1 = np.abs(jarak_1)
    jarak_1 /= np.sqrt(v1[0,0]**2 + v1[0,1]**2 + v1[0,2]**2)
    
    jarak_2 = np.sum(v2[0,0]*titik[0] + v2[0,1]*titik[1] + v2[0,2]*titik[2] - c2[0])
    jarak_2 = np.abs(jarak_2)
    jarak_2 /= np.sqrt(v2[0,0]**2 + v2[0,1]**2 + v2[0,2]**2)
    
    if jarak_1 == 0 and jarak_2 == 0:
        return 0

    if jarak_1 != 0 and jarak_2 != 0:
        return -1

    return max(jarak_1, jarak_2)

In [47]:
jarak = jarak_bidang(A1, A2, B1, B2, titik_sample)

if jarak == 0:
    print("Kedua bidang saling berimpit")
elif jarak == -1:
    print("Titik sample salah!!!")
else:
    print(f"Jarak bidang = {jarak:.2f}")

Jarak bidang = 4.52


### Cara Otomatis

In [48]:
def jarak_bidang_auto(v1, v2, c1, c2, titik):
    jarak_1 = abs(np.dot(v1, titik) - c1) / np.linalg.norm(v1)
    jarak_2 = abs(np.dot(v2, titik) - c2) / np.linalg.norm(v2)
    
    if jarak_1 == 0 and jarak_2 == 0:
        return 0

    if jarak_1 != 0 and jarak_2 != 0:
        return -1

    return max(jarak_1, jarak_2)

In [49]:
jarak_auto = jarak_bidang_auto(A1, A2, B1, B2, titik_sample)

if jarak_auto == 0:
    print("Kedua bidang saling berimpit")
elif jarak_auto == -1:
    print("Titik sample salah!!!")
else:
    print(f"Jarak bidang = {jarak_auto.item():.2f}")

Jarak bidang = 4.52


## Mencari kombinasi linear dan menentukan nilai K

In [50]:
A = np.array([[7,-4,7]])
B = np.array([[8,5,-2]])
C = np.array([[-4,-5,-6]])
Z = np.array([[-34,14,-86]])

matriks = np.concatenate((A.T, B.T, C.T, Z.T), axis=1).astype(np.float64)
matriks

array([[  7.,   8.,  -4., -34.],
       [ -4.,   5.,  -5.,  14.],
       [  7.,  -2.,  -6., -86.]])

### Cara Manual

catatan: satu utama berurutan secara diagonal dari kiri atas ke kanan bawah

In [52]:
def print_iterasi(i, matriks):
    print(f"Iterasi {i}:\n")
    print(matriks)
    print("\n")

In [53]:
i = 0
satu_utama_0 = False
satu_utama_1 = False
satu_utama_2 = False

if matriks[0,0] != 0:
    matriks[0,:] = np.round(matriks[0,:] * (1/matriks[0,0]), decimals=2)
    i += 1
    satu_utama_0 = True
    print_iterasi(i, matriks)

if matriks[1,0] != 0 and satu_utama_0:
    matriks[1,:] = np.round(matriks[0,:] * (-matriks[1,0]) + matriks[1,:], decimals=2)
    i += 1
    print_iterasi(i, matriks)

if matriks[2,0] != 0 and satu_utama_0:
    matriks[2,:] = np.round(matriks[0,:] * (-matriks[2,0]) + matriks[2,:], decimals=2)
    i += 1
    print_iterasi(i, matriks)

if matriks[1,1] != 0:
    matriks[1,:] = np.round(matriks[1,:] * (1/matriks[1,1]), decimals=2)
    i += 1
    satu_utama_1 = True
    print_iterasi(i, matriks)

if matriks[2,1] != 0 and satu_utama_1:
    matriks[2,:] = np.round(matriks[1,:] * (-matriks[2,1]) + matriks[2,:], decimals=2)
    i += 1
    print_iterasi(i, matriks)

if matriks[2,2] != 0:
    matriks[2,:] = np.round(matriks[2,:] * (1/matriks[2,2]), decimals=2)
    i += 1
    satu_utama_2 = True
    print_iterasi(i, matriks)

if matriks[1,2] != 0 and satu_utama_2:
    matriks[1,:] = np.round(matriks[2,:] * (-matriks[1,2]) + matriks[1,:], decimals=2)
    i += 1
    print_iterasi(i, matriks)

if matriks[0,2] != 0 and satu_utama_2:
    matriks[0,:] = np.round(matriks[2,:] * (-matriks[0,2]) + matriks[0,:], decimals=2)
    i += 1
    print_iterasi(i, matriks)

if matriks[0,1] != 0 and satu_utama_1:
    matriks[0,:] = np.round(matriks[1,:] * (-matriks[0,1]) + matriks[0,:], decimals=2)
    i += 1
    print_iterasi(i, matriks)

Iterasi 1:

[[  1.     1.14  -0.57  -4.86]
 [ -4.     5.    -5.    14.  ]
 [  7.    -2.    -6.   -86.  ]]


Iterasi 2:

[[  1.     1.14  -0.57  -4.86]
 [  0.     9.56  -7.28  -5.44]
 [  7.    -2.    -6.   -86.  ]]


Iterasi 3:

[[  1.     1.14  -0.57  -4.86]
 [  0.     9.56  -7.28  -5.44]
 [  0.    -9.98  -2.01 -51.98]]


Iterasi 4:

[[  1.     1.14  -0.57  -4.86]
 [  0.     1.    -0.76  -0.57]
 [  0.    -9.98  -2.01 -51.98]]


Iterasi 5:

[[  1.     1.14  -0.57  -4.86]
 [  0.     1.    -0.76  -0.57]
 [  0.     0.    -9.59 -57.67]]


Iterasi 6:

[[ 1.    1.14 -0.57 -4.86]
 [ 0.    1.   -0.76 -0.57]
 [-0.   -0.    1.    6.01]]


Iterasi 7:

[[ 1.    1.14 -0.57 -4.86]
 [ 0.    1.    0.    4.  ]
 [-0.   -0.    1.    6.01]]


Iterasi 8:

[[ 1.    1.14  0.   -1.43]
 [ 0.    1.    0.    4.  ]
 [-0.   -0.    1.    6.01]]


Iterasi 9:

[[ 1.    0.    0.   -5.99]
 [ 0.    1.    0.    4.  ]
 [-0.   -0.    1.    6.01]]




### Cara Otomatis

In [54]:
matriks_persamaan = matriks[:, :3]
matriks_jawaban = matriks[:, 3]

matriks_auto = np.linalg.solve(matriks_persamaan, matriks_jawaban)
matriks_auto

array([-5.99,  4.  ,  6.01])